In [ ]:
#TODO : REFACTOR
import numpy as np
np.random.seed(42)
import sklearn
import os
#import helper
import keras
import pickle
import math
from keras.layers.advanced_activations import LeakyReLU 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from skimage import exposure

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten,Lambda
from keras.layers import Convolution2D, MaxPooling2D, Conv2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import matplotlib.pyplot as plt
import random
import statistics
%matplotlib inline
import pandas as pd

from tqdm import tqdm

from keras.regularizers import l2

import random
import csv
from tensorflow.contrib.layers import flatten
from skimage import transform as transf

import loader
import preprocess
import plotter


#from prettytable import PrettyTable
%matplotlib inline

SEED = 22

In [ ]:
X_train, y_train, X_test, y_test,X_val,y_val = loader.load_split_input_data()
X_train_g, X_test_g,X_val_g = preprocess.convert_to_grayscale(X_train, X_test,X_val)


In [ ]:

X_train_h, X_test_h, X_val_h = preprocess.hist_equalize_set(X_train_g, X_test_g, X_val_g)
#X_train_n, X_test_n,X_val_n = preprocess.noralize(X_train_g, X_test_g,X_val_g)
X_train_hae ,X_test_hae, X_val_hae = preprocess.adaptive_equalize_set(X_train_h, X_test_h, X_val_h)
X_train_hcs,X_test_hcs,X_val_hcs = preprocess.contrast_stretch_set(X_train_h,X_test_h,X_val_h)


In [3]:
X_train_h,y_train,X_test_h,y_test,X_val_h,y_val,X_aug_h,y_aug_h = loader.load_preprocessed_data("./preprocessed/preprocessed_hist_gray.p")
X_train_hae,y_train,X_test_hae,y_test,X_val_hae,y_val,X_aug_hae,y_aug_hae = loader.load_preprocessed_data("./preprocessed/preprocessed_hae_gray.p")
X_train_hcs,y_train,X_test_hcs,y_test,X_val_hcs,y_val,X_aug_hcs,y_aug_hcs = loader.load_preprocessed_data("./preprocessed/preprocessed_hcs_gray.p")


./preprocessed/preprocessed_hist_gray.p
X_train:  34799
y_train:  34799
X_val:  4410
y_val:  4410
X_test:  12630
y_test:  12630
X_aug:  8600
y_aug:  8600
Data and modules loaded.
./preprocessed/preprocessed_hae_gray.p
X_train:  34799
y_train:  34799
X_val:  4410
y_val:  4410
X_test:  12630
y_test:  12630
X_aug:  8600
y_aug:  8600
Data and modules loaded.
./preprocessed/preprocessed_hcs_gray.p
X_train:  34799
y_train:  34799
X_val:  4410
y_val:  4410
X_test:  12630
y_test:  12630
X_aug:  8600
y_aug:  8600
Data and modules loaded.


In [4]:
def combine_datasets(datasets):

    x=datasets[0][0]
    y = datasets[0][1]
    
    if type(x) is list:
            x = np.asarray(x)
            print(str(count)+' is a list')
    if type(y) is list:
            y = np.asarray(y)
        
    #print(x.shape)
    #print(y.shape)
    X_all=x
    y_all=y
    print('combining dataset '+str(0)+' of type '+str(type(x))+' and length '+str(len(x)))
            
    count = 0
    for ds in datasets:
        if count==0: #since we added the first one already
            count+=1
            continue
       
        #print(type(ds))
        x = ds[0]
        y=ds[1]
        print('combining dataset '+str(count)+' of type '+str(type(x))+' and length '+str(len(x)))
        if type(x) is list:
            x = np.asarray(x)
            print('Dataset '+str(count)+' is a list; converting to  ndarray')
        if type(y) is list:
            y = np.asarray(y)
        
        

        X_all=np.append(X_all,x,axis=0)
        y_all=np.append(y_all,y,axis=0)
        count+=1


   
    assert(len(X_all) == len(y_all))
    print('length of combined set : '+str(len(X_all)))
    return np.asarray(X_all),np.asarray(y_all)


    
    
    
#X_all,y_all=combine_datasets([(X_train_h,y_train),(X_train_hae,y_train),(X_train_hcs,y_train)(X_train_n,y_train)])
X_all,y_all=combine_datasets([(X_train_h,y_train),(X_train_hae,y_train),(X_train_hcs,y_train)])
     

combining dataset 0 of type <class 'numpy.ndarray'> and length 34799
combining dataset 1 of type <class 'numpy.ndarray'> and length 34799
combining dataset 2 of type <class 'list'> and length 34799
Dataset 2 is a list; converting to  ndarray
length of combined set : 104397


In [ ]:
def data_augment(X,y,n_classes,n_gen_per_class,n_aug_per_class):

    print('Generating '+str(n_gen_per_class)+' number of images per class, selecting '+str(n_aug_per_class)+' number of images per class for  augmentation')
    
    datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    fill_mode='nearest',
   # horizontal_flip=True,
    )

    
    img_shape = [X.shape[1], X.shape[2], X.shape[3]]
    img_shape.insert(0,0)
    
    data_shape = np.asarray(img_shape)
    total_image_per_class = n_gen_per_class
    
    X_augmented =np.empty(data_shape)
    y_augmented =np.empty(0,dtype='uint8')
    


    print('Augmenting  Data...')
    for i in tqdm(range(n_classes)):
    #for i in tqdm(range(3)):
            #print(i)
            index = [y==i]
            images_for_i_class = X[y==i]
            y_i_class = y[y==i]
            X_augmented_i = np.empty(data_shape)
            #print(X_augmented_i.shape)
            y_augmented_i = np.empty(0,dtype='uint8')
            for X_b,y_b in datagen.flow(images_for_i_class, y_i_class, batch_size=len(y_i_class), seed=9345+i*37):            
                X_augmented_i = np.append(X_augmented_i, X_b, axis=0)
                y_augmented_i = np.append(y_augmented_i, y_b, axis=0)

                if len(X_augmented_i) >= total_image_per_class:
                    break
            X_augmented_i, y_augmented_i = shuffle(X_augmented_i, y_augmented_i, random_state=9345)
            X_augmented = np.append(X_augmented, X_augmented_i[:n_aug_per_class], axis=0)
            y_augmented = np.append(y_augmented, y_augmented_i[:n_aug_per_class], axis=0)     
    print("shufle")
    X_augmented, y_augmented = shuffle(X_augmented, y_augmented, random_state=9345)
    print("X_augmented shape: "+str(X_augmented.shape))
    print("y_augmented shape: "+str(y_augmented.shape))
    return X_augmented, y_augmented
    
    # Storing for checkpoint2
    #X_augmented = X_augmented.astype('float32')

    


In [6]:


X_aug_all, y_aug_all = preprocess.data_augment(X_all,y_all,n_classes=43,n_gen_per_class=1000,n_aug_per_class=200)
print(X_aug_all.shape)
print(y_aug_all.shape)


Generating 1000 number of images per class, selecting 200 number of images per class for  augmentation
Augmenting  Data...



100%|██████████| 43/43 [00:25<00:00,  2.93it/s]


shufle
X_augmented shape: (8600, 32, 32, 1)
y_augmented shape: (8600,)
(8600, 32, 32, 1)
(8600,)


In [15]:
from imp import reload

loader = reload(loader)

loader.save_to_disk(X_all,y_all,X_test_hae,y_test,X_val_hae,y_val,X_aug_all,y_aug_all,False,"hset_aug")

#save_aug_all_to_disk(X_aug_all,y_aug_all)

preprocessed/preprocessed_hset_aug_gray.p
Saving data to preprocessed/preprocessed_hset_aug_gray.p file...
Data cached in preprocessed/preprocessed_hset_aug_gray.p


In [16]:
X_all,y_all,X_test_hae,y_test,X_val_hae,y_val,X_aug_all,y_aug_all = loader.load_preprocessed_data("./preprocessed/preprocessed_hset_aug_gray.p")


./preprocessed/preprocessed_hset_aug_gray.p
X_train:  104397
y_train:  104397
X_val:  4410
y_val:  4410
X_test:  12630
y_test:  12630
X_aug:  8600
y_aug:  8600
Data and modules loaded.


In [17]:
X_aug_all,y_aug_all=combine_datasets([(X_all,y_all),(X_aug_all,y_aug_all)])




combining dataset 0 of type <class 'numpy.ndarray'> and length 104397
combining dataset 1 of type <class 'numpy.ndarray'> and length 8600
length of combined set : 112997
Augmented Combined Datasets:112997
Combined Datasets:104397


In [ ]:
#y_train_e = np_utils.to_categorical(y_train,num_classes=43)

n_classes = 43

y_all_e = np_utils.to_categorical(y_all,num_classes=n_classes)
y_aug_all_e = np_utils.to_categorical(y_aug_all,num_classes=n_classes)
y_test_e = np_utils.to_categorical(y_test, num_classes=n_classes)
y_val_e = np_utils.to_categorical(y_val, num_classes=n_classes)

